# Задание 1. Прямые методы решения линейных систем

In [4]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List
import warnings
warnings.filterwarnings('ignore')

In [75]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        m = self.copy()
        return latexify(self)

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

def print_markdown(*argv):
    """Displays a Markdown string"""
    res = ' '.join(['$$'] + [arg for arg in argv] + ['$$'])
    display(Markdown(res))

def get_latex_column(arg: str, n: int) -> str:
    """Prepares a string with LaTex syntax for a columt of {arg} with indeces form 1 to {n}"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


In [63]:
s = get_latex_column("x", 5)
print_markdown(s, s)

$$ \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}\\
x_{5}
\end{pmatrix} $$

## 11.1. Схема Гаусса единственного деления

Пусть требуется найти решение системы $(1) :$
$\begin{cases}
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n}x_n = a_{1n+1} \\
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n}x_n = a_{2n+1} \\
\dots \\
a_{n1}x_1 + a_{n2}x_2 + \dots + a_{nn}x_n = a_{nn+1}
\end{cases}$

Альтарнативный вид системы уравнений $:$
$\begin{pmatrix} a_{11} & a_{12} & \dots & a_{1n} \\ a_{21} & a_{22} & \dots & a_{2n} \\ \dots & \dots & \ddots & \vdots \\ a_{n1} & a_{n2} & \dots & a_{nn} \end{pmatrix} \begin{pmatrix} x_1 \\ x_2 \\ \vdots \\ x_n \end{pmatrix} = \begin{pmatrix} a_{1n+1} \\ a_{2n+1} \\ \vdots \\ a_{nn+1} \end{pmatrix} $

In [79]:
n = int(input()) # Размерность матрицы A
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_markdown(A.tex(), get_latex_column('x', n), '=', B.tex())

$$ \begin{pmatrix} 7.439141 & 1.023102 & 8.995423 & 2.647606 \\ 7.412118 & 7.183387 & 0.259968 & 5.015531 \\ 8.560962 & 0.851391 & 0.786663 & 7.815219 \\ 3.187618 & 2.405368 & 7.071941 & 9.653040 \end{pmatrix} \begin{pmatrix}
x_{1}\\
x_{2}\\
x_{3}\\
x_{4}
\end{pmatrix} = \begin{pmatrix} 5.494232 \\ 2.206854 \\ 0.174839 \\ 2.485316 \end{pmatrix} $$

### Решение методом Гаусса единственного деления

In [111]:
def SingleGauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    A_ = A.copy()
    B_ = B.copy()
    X = [[0] for _ in range(n)]
    for k in range(n):
        for j in range(k+1, n):
            d = A_[j][k] / A_[k][k]
            for i in range(k, n):
                A_[j][i] = A_[j][i] - d * A_[k][i]
            B_[j] -= d * B_[k][0]
    for k in range(n-1, -1, -1):
        d = 0
        for j in range(k, n):
            s = A_[k][j] * X[j][0]
            d += s
        X[k][0] = (B_[k][0] - d) / A_[k][k]
    return Matrix(np.array(X))


X = SingleGauss(A, B, n)
print_markdown('X^* = ', get_latex_column("x^*", n), '=', X.tex())
print_markdown(r'A \times X^* = ', A.tex(), X.tex(), '=', (np.matmul(A, X)).tex(), '= B^*')
print_markdown('B = ', B.tex(), r'\stackrel{?}{=}', (np.matmul(A, X)).tex(), '= B^*')
max(B - np.matmul(A, X))


$$ X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 0.167571 \\ 0.286047 \\ 0.511434 \\ -0.243831 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 7.439141 & 1.023102 & 8.995423 & 2.647606 \\ 7.412118 & 7.183387 & 0.259968 & 5.015531 \\ 8.560962 & 0.851391 & 0.786663 & 7.815219 \\ 3.187618 & 2.405368 & 7.071941 & 9.653040 \end{pmatrix} \begin{pmatrix} 0.167571 \\ 0.286047 \\ 0.511434 \\ -0.243831 \end{pmatrix} = \begin{pmatrix} 5.494232 \\ 2.206854 \\ 0.174839 \\ 2.485316 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix} 5.494232 \\ 2.206854 \\ 0.174839 \\ 2.485316 \end{pmatrix} \stackrel{?}{=} \begin{pmatrix} 5.494232 \\ 2.206854 \\ 0.174839 \\ 2.485316 \end{pmatrix} = B^* $$

Matrix([1.11022302e-15])